In [3]:
import folium
import pandas as pd
import ast

In [4]:
# load network
network = pd.read_csv('outputs/osmnx-df.csv')
network['start_coords'] = network['start_coords'].apply(ast.literal_eval)
network['end_coords'] = network['end_coords'].apply(ast.literal_eval)

In [5]:
trips = pd.read_csv('outputs/trips.csv', index_col=0)
train_data = pd.read_csv('outputs/train.csv', index_col=0)


In [22]:
trip = trips.iloc[0]
data = train_data[train_data.cab_id == trip.cabID]

relevant_gps_data = data.iloc[trip['tripStartIDX']:trip['tripEndIDX'] + 1]
relevant_gps_data = relevant_gps_data[['latitude', 'longitude', 'time']]



In [27]:
routes = [5479, 5480, 6907, 6908, 7131, 7132, 7833, 11856, 11960, 11961, 12090, 5479, 5480, 6907, 6908, 6909, 6910, 6912, 11960, 11961, 12089, 12090, 12092, 7316, 4734, 4735, 4736, 4737, 4738, 5481, 5482, 5483, 6302, 6684, 6685, 6686, 6687, 6688, 6690, 7123, 7124, 7133, 7441, 7497, 7831, 8876, 8877, 9200, 9201, 9202, 9203, 9205, 10299, 10848, 10849, 12257, 4536, 4538, 4539, 4540, 4541, 4542, 4543, 4544, 4739, 4740, 4741, 6243, 6309, 6806, 7459, 7460, 7461, 11987, 7503, 7109, 5983, 6630, 7152, 9106, 9108, 9261, 9262, 9263, 9307, 9308, 9309, 9310, 9311, 10683, 11906, 4188, 4189, 4190, 4191, 4192, 4193, 4194, 4195, 4196, 4197, 4198, 4199, 5556, 5557, 5558, 7584, 12281, 12282, 12283, 12284, 12285, 12286, 4189, 4190, 4191, 4192, 4193, 4194, 4195, 4196, 4197, 4198, 4199, 7528, 4204, 4205, 4206, 4207, 4208, 9925, 10013, 11638, 4242, 4916, 4917, 4918, 5948, 5949, 11706, 4916, 4917, 4918, 4919, 4920, 4921, 4922, 4923, 5275, 5276, 5277, 5313, 5477, 5942, 5948, 5949, 9372, 9373, 9374, 9392, 9407, 9419, 9421, 11172, 11706, 12380, 4921, 4922, 4923, 5477, 5942, 9373]

routes = list(set(routes))

In [31]:
def san_francisco_bounding_box():
  # San Francisco coordinates
  sf_lat, sf_lon = 37.7749, -122.4194

  # Define buffer in degrees of latitude and longitude
  buffer_degrees = 0.2

  # Calculate bounding box coordinates
  min_lat = sf_lat - buffer_degrees - 0.25
  max_lat = sf_lat + buffer_degrees - 0.02
  min_lon = sf_lon - buffer_degrees
  max_lon = sf_lon + buffer_degrees + 0.25

  # Return bounding box as a list of coordinates
  return [[min_lat, min_lon], [max_lat, max_lon]] #actual box for visualizaiton folium
  # return box(float(min_lon), float(min_lat), float(max_lon), float(max_lat))
  # return Polygon([(min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat)]) #actual bounding box for filtering


def map_visualization(data, network):

    # Create a map centered on San Francisco
    map = folium.Map(location=[37.7749, -122.4194], zoom_start=13)

    # Add the bounding box to the map as a rectangle
    folium.Rectangle(bounds=san_francisco_bounding_box(), color='red', fill=False).add_to(map)

    # Add the road network to the map as a set of polylines
    for _, row in network.iterrows():
        start_coords = row['start_coords']
        end_coords = row['end_coords']
        polyline = folium.PolyLine(locations=[start_coords, end_coords], color="red", weight=9)
        map.add_child(polyline)

    # Add markers for each GPS point to the map
    for _, row in data.iterrows():
        coords = [row["latitude"], row["longitude"]]
        # popup = f"Occupancy: {row['occupancy']}, Time: {row['time']}"
        circle_marker = folium.CircleMarker(location=coords, radius=10, color="blue", fill=True, fill_color="blue")
        circle_marker.add_to(map)

    # Display the map
    return map

modified_network = network.loc[routes]
m = map_visualization(relevant_gps_data.iloc[0:3], modified_network)
m


In [25]:
network[network.road_name == 'Pine Street']

,edge_id,start_coords,end_coords,osmid,start_node,end_node,oneway,lanes,ref,road_name,highway,maxspeed,reversed,length,geometry
4537,65292738.0-65328348.0,"(37.7883758, -122.4303739)","(37.7881668, -122.4320179)",7870678,6.529274e+07,6.532835e+07,True,NaN,NaN,Pine Street,secondary,20 mph,False,146.324,"LINESTRING (-122.4303739 37.7883758, -122.4304..."
5086,65301554.0-65336823.0,"(37.786903, -122.4419331)","(37.7866945, -122.4435748)",196949610,6.530155e+07,6.533682e+07,True,NaN,NaN,Pine Street,secondary,20 mph,False,146.119,"LINESTRING (-122.4419331 37.786903, -122.44202..."
5351,65305817.0-65352429.0,"(37.791909, -122.4025983)","(37.7917031, -122.4042334)",254759910,6.530582e+07,6.535243e+07,True,NaN,NaN,Pine Street,secondary,20 mph,False,145.492,"LINESTRING (-122.4025983 37.791909, -122.40272..."
6245,65319030.0-65292738.0,"(37.7885845, -122.4287315)","(37.7883758, -122.4303739)",8917360,6.531903e+07,6.529274e+07,True,NaN,NaN,Pine Street,secondary,20 mph,False,146.181,"LINESTRING (-122.4287315 37.7885845, -122.4288..."
6658,65326736.0-65343967.0,"(37.7908839, -122.4106394)","(37.7906745, -122.4122872)",26965876,6.532674e+07,6.534397e+07,True,2.0,NaN,Pine Street,secondary,20 mph,False,146.656,"LINESTRING (-122.4106394 37.7908839, -122.4107..."
6685,65327169.0-258758546.0,"(37.7896296, -122.4205089)","(37.7894339, -122.4220476)",408600295,6.532717e+07,2.587585e+08,True,4.0,NaN,Pine Street,secondary,20 mph,False,136.951,"LINESTRING (-122.4205089 37.7896296, -122.4206..."
6704,65327403.0-65336474.0,"(37.7924133, -122.3986103)","(37.7922611, -122.3997849)",111089307,6.532740e+07,6.533647e+07,True,2.0,NaN,Pine Street,secondary,20 mph,False,104.592,"LINESTRING (-122.3986103 37.7924133, -122.3987..."
6755,65328348.0-65350912.0,"(37.7881668, -122.4320179)","(37.788062, -122.432843)",27166421,6.532835e+07,6.535091e+07,True,NaN,NaN,Pine Street,secondary,20 mph,False,73.436,"LINESTRING (-122.4320179 37.7881668, -122.4321..."
6789,65328705.0-65352435.0,"(37.7915047, -122.4057597)","(37.7913012, -122.4073583)",254299130,6.532870e+07,6.535244e+07,True,2.0,NaN,Pine Street,secondary,20 mph,False,142.283,"LINESTRING (-122.4057597 37.7915047, -122.4060..."
6806,65329111.0-65351407.0,"(37.7890027, -122.4254412)","(37.7887934, -122.4270878)",514440706,6.532911e+07,6.535141e+07,True,NaN,NaN,Pine Street,secondary,20 mph,False,146.554,"LINESTRING (-122.4254412 37.7890027, -122.4255..."


In [48]:
def san_francisco_bounding_box():
  # San Francisco coordinates
  sf_lat, sf_lon = 37.7749, -122.4194

  # Define buffer in degrees of latitude and longitude
  buffer_degrees = 0.2

  # Calculate bounding box coordinates
  min_lat = sf_lat - 0.45
  max_lat = sf_lat + 0.2
  min_lon = sf_lon - 0.2
  max_lon = sf_lon + 0.45

  # Return bounding box as a list of coordinates
  return [[min_lat, min_lon], [max_lat, max_lon]]

def map_visualization(data, network):

    # Create a map centered on San Francisco
    map = folium.Map(location=[37.7749, -122.4194], zoom_start=13)

    # Add the bounding box to the map as a rectangle
    folium.Rectangle(bounds=san_francisco_bounding_box(), color='red', fill=False).add_to(map)

    # Add the road network to the map as a set of polylines
    for _, row in network.iterrows():
        start_coords = row['start_coords']
        end_coords = row['end_coords']
        polyline = folium.PolyLine(locations=[start_coords, end_coords], color="red", weight=9)
        map.add_child(polyline)

    # Add markers for each GPS point to the map
    for _, row in data.iterrows():
        coords = [row["latitude"], row["longitude"]]
        # popup = f"Occupancy: {row['occupancy']}, Time: {row['time']}"
        circle_marker = folium.CircleMarker(location=coords, radius=10, color="blue", fill=True, fill_color="blue")
        circle_marker.add_to(map)

    # Display the map
    return map

routes = pd.read_csv('old_network_map_matched.csv')

modified_network = network.loc[routes['road_segment'].tolist()]

m = map_visualization(relevant_gps_data, modified_network)
m

In [49]:
routes['road_segment'].tolist()

[5351,
 6789,
 7494,
 6302,
 6309,
 7503,
 10684,
 10224,
 6629,
 7529,
 9926,
 4242,
 5477,
 4644]

In [32]:
from mapMatching.helpers import get_closest_point, get_distance
min_dists = []
rel_network = network[network.road_name == 'Pine Street']
for idx, row in relevant_gps_data.iloc[0:3].iterrows():
    min_dist = 1000000
    gps_point = row
    for idx, row in rel_network.iterrows():
        closest_point = get_closest_point(gps_point, row)
        distance = get_distance(gps_point, closest_point)
        min_dist = min(distance, min_dist)
    min_dists.append(min_dist)

print(min_dists)

[0.07855526548631712, 0.002547005575181982, 0.0008428000941810724]


hmm distances are super close, could be an issue with the geodataframe. Let me try just bounding the network first and trying regular HMM on that and see if it's too slow. If it is we can try a mix. Not sure why these roads weren't selected normally, and not sure why k-closest is returning a whole bunch.

In [26]:
def san_francisco_bounding_box():
  # San Francisco coordinates
  sf_lat, sf_lon = 37.7749, -122.4194

  # Define buffer in degrees of latitude and longitude
  buffer_degrees = 0.2

  # Calculate bounding box coordinates
  min_lat = sf_lat - buffer_degrees - 0.25
  max_lat = sf_lat + buffer_degrees - 0.02
  min_lon = sf_lon - buffer_degrees
  max_lon = sf_lon + buffer_degrees + 0.25

  # Return bounding box as a list of coordinates
  return [[min_lat, min_lon], [max_lat, max_lon]] #actual box for visualizaiton folium
  # return box(float(min_lon), float(min_lat), float(max_lon), float(max_lat))
  # return Polygon([(min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat)]) #actual bounding box for filtering


def map_visualization(data, network):

    # Create a map centered on San Francisco
    map = folium.Map(location=[37.7749, -122.4194], zoom_start=13)

    # Add the bounding box to the map as a rectangle
    # folium.Rectangle(bounds=san_francisco_bounding_box(), color='red', fill=False).add_to(map)

    # Add the road network to the map as a set of polylines
    for _, row in network.iterrows():
        start_coords = row['start_coords']
        end_coords = row['end_coords']
        polyline = folium.PolyLine(locations=[start_coords, end_coords], color="red", weight=4)
        map.add_child(polyline)

    # Add markers for each GPS point to the map
    for _, row in data.iterrows():
        coords = [row["latitude"], row["longitude"]]
        # popup = f"Occupancy: {row['occupancy']}, Time: {row['time']}"
        circle_marker = folium.CircleMarker(location=coords, radius=10, color="blue", fill=True, fill_color="blue")
        circle_marker.add_to(map)

    # Display the map
    return map

modified_network =network[network.road_name == 'Pine Street']
m = map_visualization(relevant_gps_data, modified_network)
m
